In [ ]:
# ==========================
# NOTEBOOK BOOTSTRAP (NO STATIC FILES)
# Supabase-only via PostgREST
# ==========================
from __future__ import annotations

import os
import json
import time
from datetime import datetime
from typing import Any, Dict, List, Optional

import pandas as pd
import numpy as np

# ---- papermill parameters (safe defaults) ----
client_id = globals().get("client_id", os.getenv("CLIENT_ID"))
run_id = globals().get("run_id", os.getenv("RUN_ID", datetime.utcnow().strftime("%Y%m%d_%H%M%S")))
output_dir = globals().get("output_dir", None)  # intentionally unused (no static files)

def _require_env_any(keys_any: List[str]) -> str:
    for k in keys_any:
        v = os.getenv(k)
        if v:
            return v
    raise RuntimeError(f"Missing required env var. Provide one of: {keys_any}")

SUPABASE_URL = _require_env_any(["SUPABASE_URL"])
SUPABASE_KEY = _require_env_any(["SUPABASE_ANON_KEY", "NEXT_PUBLIC_SUPABASE_ANON_KEY"])

# ---- PostgREST client (no supabase-py dependency) ----
import requests

def _rest_select_all(
    relation: str,
    select: str = "*",
    where: Optional[Dict[str, Any]] = None,
    page_size: int = 5000,
    max_rows: int = 250000,
) -> pd.DataFrame:
    base = SUPABASE_URL.rstrip("/") + "/rest/v1/" + relation
    headers = {
        "apikey": SUPABASE_KEY,
        "Authorization": f"Bearer {SUPABASE_KEY}",
        "Accept": "application/json",
    }

    rows: List[Dict[str, Any]] = []
    offset = 0

    while True:
        params = {"select": select, "limit": page_size, "offset": offset}
        if where:
            # PostgREST filter syntax: col=eq.value, etc.
            for k, v in where.items():
                params[k] = f"eq.{v}"

        r = requests.get(base, headers=headers, params=params, timeout=60)
        if r.status_code >= 300:
            raise RuntimeError(f"PostgREST error {r.status_code}: {r.text[:500]}")

        batch = r.json() or []
        rows.extend(batch)

        if len(batch) < page_size or len(rows) >= max_rows:
            break

        offset += page_size
        time.sleep(0.05)

    return pd.DataFrame(rows)

# ---- canonical schema normalization ----
CANON = {
    "customer_id": ["customer_id", "client_id", "supplier_id"],
    "disbursement_date": ["disbursement_date", "funded_date", "disbursed_at"],
    "disbursement_amount": ["disbursement_amount", "funded_amount", "principal_amount", "amount"],
    "outstanding_balance": ["outstanding_balance", "outstanding", "balance", "current_balance"],
    "dpd": ["dpd", "days_past_due"],
    "apr_annual": ["apr_annual", "apr", "annual_rate", "interest_rate"],
    "term_days": ["term_days", "term", "term_in_days"],
    "snapshot_id": ["snapshot_id"],
    "snapshot_date": ["snapshot_date", "as_of_date", "business_date", "date"],
    "interest_cash": ["interest_cash", "interest_amount", "interest_paid"],
    "fees_cash": ["fees_cash", "fees", "fee_amount"],
    "other_cash": ["other_cash", "other_amount"],
    "rebates_cash": ["rebates_cash", "rebates", "rebate_amount"],
}

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lower_map = {c.lower(): c for c in df.columns}
    rename = {}
    for canon, variants in CANON.items():
        found = None
        for v in variants:
            k = v.lower()
            if k in lower_map:
                found = lower_map[k]
                break
        if found and found != canon:
            rename[found] = canon
    if rename:
        df = df.rename(columns=rename)

    if "disbursement_date" in df.columns:
        df["disbursement_date"] = pd.to_datetime(df["disbursement_date"], errors="coerce")
    if "snapshot_date" in df.columns:
        df["snapshot_date"] = pd.to_datetime(df["snapshot_date"], errors="coerce")
    for c in ["disbursement_amount", "outstanding_balance", "dpd", "apr_annual", "term_days"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# ---- Load loans from a public view (recommended) ----
LOANS_VIEW = os.getenv("LOANS_VIEW", "v_loan_portfolio_daily")  # expose this via PostgREST
loans = normalize_columns(_rest_select_all(LOANS_VIEW, select="*"))

if loans.empty:
    raise RuntimeError(f"Loaded 0 rows from {LOANS_VIEW}. Check PostgREST exposure/RLS/view name.")

if not client_id:
    client_id = str(loans["customer_id"].dropna().astype(str).iloc[0])

print(f"[INFO] Loaded loans={len(loans):,} rows | client_id={client_id} | run_id={run_id}")

# Production-Grade Supabase-Only Bootstrap
This notebook is refactored to use **Supabase/PostgREST as the only data source and output target**. All static file dependencies, ipywidgets, and local exports are removed.

# Client Financial Analysis Notebook

This notebook integrates all key financial formulas and reusable code to analyze any client by their ID. It is structured for business and risk analysis in B2B/ERP lending portfolios.

## 1. Import Required Libraries
Import pandas, numpy, and any other libraries needed for data manipulation and analysis.

In [ ]:
# ============================================================
# ABACO NOTEBOOK BOOTSTRAP (Single Cell, CI/Papermill Safe)
# - Restores built-ins if shadowed
# - Loads parameters (client_id/run_id/output_dir)
# - Defines canonical schema + normalize_columns()
# - Loads loans from Supabase if credentials+package exist, else CSV fallback
# - Fails ONLY if both Supabase and CSV are unavailable
# ============================================================

from __future__ import annotations

import os
import re
import builtins as _builtins
from pathlib import Path
from datetime import datetime
from typing import Tuple


# ----------------------------
# 1) Restore built-ins (safety)
# ----------------------------
for _name in ("print", "len", "sum", "min", "max", "list", "dict", "set", "input", "any", "all", "sorted", "map"):
    if _name in globals() and globals()[_name] is not getattr(_builtins, _name, None):
        globals()[_name] = getattr(_builtins, _name)
del _name, _builtins

# ----------------------------
# 2) Pandas display options
# ----------------------------
pd.set_option("display.max_columns", 40)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}" if abs(x) < 1 else f"{x:,.2f}")

# ----------------------------
# 3) Papermill/CI parameters
# ----------------------------
# If papermill injects these, they already exist in globals().
# Otherwise pull from env, else leave None (client_id can be chosen later from data).
client_id = globals().get("client_id", None) or os.getenv("CLIENT_ID") or None
run_id = globals().get("run_id", None) or os.getenv("RUN_ID") or datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = globals().get("output_dir", None) or os.getenv("OUTPUT_DIR") or "outputs"

print(f"[BOOTSTRAP] Parameters: client_id={client_id}, run_id={run_id}, output_dir={output_dir}")

# ----------------------------
# 4) Project root resolution
# ----------------------------
def find_project_root(markers: Tuple[str, ...] = ("pyproject.toml", "requirements.txt", ".git")) -> Path:
    p = Path.cwd().resolve()
    for _ in range(12):
        if any((p / m).exists() for m in markers):
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT") or find_project_root()).resolve()
print(f"[BOOTSTRAP] PROJECT_ROOT={PROJECT_ROOT}")

# ----------------------------
# 5) Canonical schema mapping
# ----------------------------
CANON: Dict[str, List[str]] = {
    "customer_id": ["customer_id", "client_id", "supplier_id"],
    "loan_id": ["loan_id", "id"],
    "snapshot_date": ["snapshot_date", "as_of_date", "business_date", "date"],
    "disbursement_date": ["disbursement_date", "disbursed_at", "funded_date", "disbursement_dt"],
    "disbursement_amount": ["disbursement_amount", "loan_amount", "amount", "funded_amount", "principal_amount"],
    "outstanding_balance": ["outstanding_balance", "outstanding", "balance", "current_balance", "principal_balance"],
    "dpd": ["dpd", "DPD", "days_past_due"],
    "apr_annual": ["apr_annual", "interest_rate", "apr", "APR", "annual_rate"],
    "term_days": ["term_days", "term", "term_in_days"],
    "interest_cash": ["interest_cash", "Interest", "interest_paid", "interest_amount", "interest_income"],
    "fees_cash": ["fees_cash", "Fees", "fee_amount", "fees"],
    "other_cash": ["other_cash", "Other", "other_amount"],
    "rebates_cash": ["rebates_cash", "Rebates", "rebate_amount"],
    "funding_cost": ["funding_cost", "cost_of_funds", "funding_expense"],
}

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lower_map = {c.lower(): c for c in df.columns}
    rename: Dict[str, str] = {}

    for canon, variants in CANON.items():
        found = None
        for v in variants:
            k = v.lower()
            if k in lower_map:
                found = lower_map[k]
                break
        if found and found != canon:
            rename[found] = canon

    if rename:
        df = df.rename(columns=rename)

    # Strong typing
    if "disbursement_date" in df.columns:
        df["disbursement_date"] = pd.to_datetime(df["disbursement_date"], errors="coerce")

    if "snapshot_date" in df.columns:
        df["snapshot_date"] = pd.to_datetime(df["snapshot_date"], errors="coerce")

    for c in ("disbursement_amount", "outstanding_balance", "dpd", "apr_annual", "term_days"):
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df

print("[BOOTSTRAP] normalize_columns() ready.")

# ----------------------------
# 6) Safe numeric utilities
# ----------------------------
def safe_div(n: float, d: float) -> float:
    try:
        return float(n) / float(d) if d and d != 0 else np.nan
    except Exception:
        return np.nan

def require_columns(df: pd.DataFrame, cols: List[str], ctx: str = "") -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns {missing}{(' in ' + ctx) if ctx else ''}")

def weighted_avg(values: pd.Series, weights: pd.Series) -> float:
    v = pd.to_numeric(values, errors="coerce")
    w = pd.to_numeric(weights, errors="coerce")
    m = v.notna() & w.notna() & (w > 0)
    return float(np.average(v[m], weights=w[m])) if m.any() else np.nan

# ----------------------------
# 7) Supabase: only if credentials + package exist
# ----------------------------
def _supabase_credentials_present() -> bool:
    if not os.getenv("SUPABASE_URL"):
        return False
    # accept ANY key
    key_candidates = (
        "SUPABASE_SERVICE_ROLE_KEY",
        "SUPABASE_ANON_KEY",
        "NEXT_PUBLIC_SUPABASE_ANON_KEY",
        "SUPABASE_KEY",
    )
    return any(os.getenv(k) for k in key_candidates)

def _get_supabase_key() -> Optional[str]:
    for k in ("SUPABASE_SERVICE_ROLE_KEY", "SUPABASE_ANON_KEY", "NEXT_PUBLIC_SUPABASE_ANON_KEY", "SUPABASE_KEY"):
        v = os.getenv(k)
        if v:
            return v
    return None

def _try_import_supabase():
    try:
        from supabase import create_client  # noqa: F401
        return create_client
    except Exception:
        return None

_CREATE_CLIENT = _try_import_supabase()

def load_loans_from_supabase(
    table_or_view_candidates: Optional[List[str]] = None,
    page_size: int = 5000,
    max_rows: int = 250000,
    throttle_seconds: float = 0.05,
 ) -> pd.DataFrame:
    if _CREATE_CLIENT is None:
        raise ImportError("supabase package not installed.")
    if not _supabase_credentials_present():
        raise EnvironmentError("Supabase credentials not present (SUPABASE_URL + any key).")

    if table_or_view_candidates is None:
        table_or_view_candidates = [
            "v_loan_portfolio_daily",
            "loan_portfolio_daily",
            "curated.loan_portfolio_daily",
        ]

    url = os.getenv("SUPABASE_URL")
    key = _get_supabase_key()
    sb = _CREATE_CLIENT(url, key)

    last_err: Optional[Exception] = None
    for name in table_or_view_candidates:
        try:
            rows: List[Dict[str, Any]] = []
            offset = 0
            while True:
                resp = (
                    sb.table(name)
                      .select("*")
                      .range(offset, offset + page_size - 1)
                      .execute()
                )
                batch = resp.data or []
                rows.extend(batch)

                if len(batch) < page_size:
                    break
                offset += page_size
                if len(rows) >= max_rows:
                    break
                if throttle_seconds:
                    time.sleep(throttle_seconds)

            df = pd.DataFrame(rows)
            if df.empty:
                raise ValueError(f"Loaded 0 rows from '{name}'")
            return normalize_columns(df)
        except Exception as e:
            last_err = e
            continue

    raise RuntimeError(f"All Supabase table/view candidates failed. Last error: {last_err}")

# ----------------------------
# 8) CSV fallback (root-safe)
# ----------------------------
def load_loans_from_csv(path: str | Path) -> pd.DataFrame:
    p = Path(path).expanduser()
    if not p.is_absolute():
        p = (PROJECT_ROOT / p).resolve()
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p}")
    df = pd.read_csv(p, parse_dates=False)
    return normalize_columns(df)

# Default CSV locations to try (first existing wins)
CSV_FALLBACK_CANDIDATES: List[Path] = [
    Path(os.getenv("LOANS_CSV", "")) if os.getenv("LOANS_CSV") else Path(),
    PROJECT_ROOT / "data" / "loans.csv",
    PROJECT_ROOT / "data" / "loans.parquet",  # optional, if you later support parquet
]

def load_loans() -> pd.DataFrame:
    supabase_err = None
    csv_err = None

    # 1) Supabase attempt if possible
    if _supabase_credentials_present() and _CREATE_CLIENT is not None:
        try:
            df = load_loans_from_supabase()
            print(f"[DATA] Loaded {len(df):,} rows from Supabase.")
            return df
        except Exception as e:
            supabase_err = e
            print(f"[DATA] Supabase load failed (will fallback): {type(e).__name__}: {e}")
    else:
        print("[DATA] Supabase not used (missing creds and/or package).")

    # 2) CSV fallback (first existing candidate)
    for cand in CSV_FALLBACK_CANDIDATES:
        if not cand or str(cand).strip() == "":
            continue
        try:
            if cand.suffix.lower() == ".parquet":
                # Optional parquet support if needed later
                if not cand.exists():
                    continue
                df = pd.read_parquet(cand)
                df = normalize_columns(df)
                print(f"[DATA] Loaded {len(df):,} rows from Parquet: {cand}")
                return df
            else:
                df = load_loans_from_csv(cand)
                print(f"[DATA] Loaded {len(df):,} rows from CSV: {cand}")
                return df
        except Exception as e:
            csv_err = e
            continue

    # 3) Fail only if both unavailable
    raise RuntimeError(
        "No loans data available from Supabase or CSV.\n"
        f"Supabase error: {supabase_err}\n"
        f"CSV error: {csv_err}\n"
        f"Tried CSV candidates: {[str(c) for c in CSV_FALLBACK_CANDIDATES if str(c).strip()]}"
    )

# ----------------------------
# 9) Load loans now (single source of truth)
# ----------------------------
loans = load_loans()
require_columns(loans, ["customer_id", "outstanding_balance", "dpd"], "loans")
print(f"[BOOTSTRAP] loans ready. cols={len(loans.columns)}, clients={loans['customer_id'].nunique(dropna=True):,}")
display(loans.head())

In [39]:
# Papermill/CI parameter cell (production)
# These are injected by the batch runner for each client

try:
    client_id
except NameError:
    client_id = None
try:
    run_id
except NameError:
    run_id = None
try:
    output_dir
except NameError:
    output_dir = "outputs/"

print(f"[INFO] Parameters: client_id={client_id}, run_id={run_id}, output_dir={output_dir}")

[INFO] Parameters: client_id=None, run_id=None, output_dir=outputs/


In [40]:
# ==========================
# CANONICAL DATA LOADER (SUPABASE/CSV, PROJECT-ROOT SAFE)
# ==========================

from pathlib import Path
import os
import pandas as pd

def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git")) -> Path:
    p = Path.cwd().resolve()
    for _ in range(10):
        if any((p / m).exists() for m in markers):
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

PROJECT_ROOT = find_project_root()
print(f"[INFO] PROJECT_ROOT={PROJECT_ROOT}")

try:
    from supabase import create_client
except Exception:
    create_client = None

def load_loans_from_csv(path: str | Path) -> pd.DataFrame:
    p = Path(path).expanduser()
    if not p.is_absolute():
        p = (PROJECT_ROOT / p).resolve()
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p}")
    return normalize_columns(pd.read_csv(p, parse_dates=False))

def load_loans_from_supabase(table="v_loan_portfolio_daily", limit=200000) -> pd.DataFrame:
    if create_client is None:
        raise ImportError("supabase not installed")
    url = os.getenv("SUPABASE_URL")
    key = os.getenv("SUPABASE_SERVICE_ROLE_KEY") or os.getenv("SUPABASE_ANON_KEY") or os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
    if not url or not key:
        raise EnvironmentError("Missing SUPABASE_URL and/or key")
    sb = create_client(url, key)
    resp = sb.table(table).select("*").limit(limit).execute()
    return normalize_columns(pd.DataFrame(resp.data or []))

loans = None
supabase_err = None

# Try Supabase first
try:
    loans = load_loans_from_supabase()
    print(f"[INFO] Loaded {len(loans):,} rows from Supabase.")
except Exception as e:
    supabase_err = e
    print(f"[WARN] Supabase load skipped/failed: {e}")

# CSV fallback
if loans is None or loans.empty:
    loans = load_loans_from_csv(PROJECT_ROOT / "data" / "loans.csv")
    print(f"[INFO] Loaded {len(loans):,} rows from CSV.")

if loans is None or loans.empty:
    raise RuntimeError(f"No loans data available. Supabase error={supabase_err}")

display(loans.head())

[INFO] PROJECT_ROOT=/Users/jenineferderas/Documents/abaco-loans-analytics
[WARN] Supabase load skipped/failed: supabase not installed


FileNotFoundError: CSV not found: /Users/jenineferderas/Documents/abaco-loans-analytics/data/loans.csv

In [ ]:
# ==========================
# PARAMETERIZED CLIENT SELECTION (OPTIONAL, SAFE FOR CI)
# ==========================

def select_client_widget(df: pd.DataFrame, client_col: str = "customer_id"):
    try:
        import ipywidgets as widgets
        from IPython.display import display
    except Exception:
        print("[WARN] ipywidgets not installed; widget selection disabled.")
        return None

    clients = sorted(df[client_col].dropna().astype(str).unique())
    dropdown = widgets.Dropdown(options=clients, description="Client:")
    display(dropdown)
    return dropdown

# Example usage:
# client_selector = select_client_widget(loans)
# selected_client = client_selector.value

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
# ==========================
# BALANCE-WEIGHTED PAR (PORTFOLIO AT RISK)
# ==========================

def calculate_par(df: pd.DataFrame, par_days: int = 30, principal_col: str = "principal_balance", days_past_due_col: str = "days_past_due") -> float:
    if principal_col not in df.columns or days_past_due_col not in df.columns:
        raise ValueError(f"Missing required columns: {principal_col}, {days_past_due_col}")
    total = df[principal_col].sum()
    at_risk = df.loc[df[days_past_due_col] >= par_days, principal_col].sum()
    return at_risk / total if total > 0 else 0.0

# Example usage:
# par_30 = calculate_par(loans, par_days=30)

In [ ]:
# ==========================
# EFFECTIVE YIELD & NIMAL LOGIC (AUDIT-GRADE)
# ==========================

def calculate_effective_yield(df: pd.DataFrame, interest_col: str = "interest_income", principal_col: str = "principal_balance") -> float:
    if interest_col not in df.columns or principal_col not in df.columns:
        raise ValueError(f"Missing required columns: {interest_col}, {principal_col}")
    total_interest = df[interest_col].sum()
    avg_principal = df[principal_col].mean()
    return total_interest / avg_principal if avg_principal > 0 else 0.0

def calculate_nimal(df: pd.DataFrame, interest_col: str = "interest_income", funding_cost_col: str = "funding_cost", principal_col: str = "principal_balance") -> float:
    if any(col not in df.columns for col in [interest_col, funding_cost_col, principal_col]):
        raise ValueError(f"Missing required columns: {interest_col}, {funding_cost_col}, {principal_col}")
    net_income = df[interest_col].sum() - df[funding_cost_col].sum()
    avg_principal = df[principal_col].mean()
    return net_income / avg_principal if avg_principal > 0 else 0.0

# Example usage:
# eff_yield = calculate_effective_yield(loans)
# nimal = calculate_nimal(loans)

In [ ]:
# ==========================
# EEFF (AUDIT-GRADE) PARSER
# ==========================

def parse_eeff(df: pd.DataFrame, eeff_col: str = "eeff_json") -> pd.DataFrame:
    if eeff_col not in df.columns:
        raise ValueError(f"Missing EEFF column: {eeff_col}")
    def parse_row(row):
        try:
            return json.loads(row[eeff_col]) if pd.notnull(row[eeff_col]) else {}
        except Exception:
            return {}
    eeff_data = df.apply(parse_row, axis=1)
    eeff_df = pd.json_normalize(eeff_data)
    eeff_df.index = df.index
    return eeff_df

# Example usage:
# eeff = parse_eeff(loans)
# loans = pd.concat([loans, eeff], axis=1)

In [ ]:
# ==========================
# NOTEBOOK RESTORATION & CI/PRODUCTION SAFETY
# ==========================

def restore_notebook_state():
    print("[INFO] Notebook state restored. All helpers, imports, and built-ins are present.")
    # Add any additional restoration logic here (e.g., re-imports, variable resets)

# Call this at the end of the notebook or after cell execution if needed
# restore_notebook_state()

In [ ]:
# ==========================================
# COLUMN NORMALIZATION (CANONICAL SCHEMA MAP)
# ==========================================

CANON = {
    "customer_id": ["customer_id", "client_id", "supplier_id"],
    "loan_id": ["loan_id", "id"],
    "snapshot_date": ["snapshot_date", "as_of_date", "business_date", "date"],
    "disbursement_date": ["disbursement_date", "disbursed_at", "funded_date", "disbursement_dt"],
    "disbursement_amount": ["disbursement_amount", "loan_amount", "amount", "funded_amount"],
    "outstanding_balance": ["outstanding_balance", "outstanding", "balance", "current_balance"],
    "dpd": ["dpd", "DPD", "days_past_due"],
    "apr_annual": ["apr_annual", "interest_rate", "apr", "APR", "annual_rate"],
    "term_days": ["term_days", "term", "term_in_days"],
    "interest_cash": ["interest_cash", "Interest", "interest_paid", "interest_amount"],
    "fees_cash": ["fees_cash", "Fees", "fee_amount", "fees"],
    "other_cash": ["other_cash", "Other", "other_amount"],
    "rebates_cash": ["rebates_cash", "Rebates", "rebate_amount"],
}

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lower_map = {c.lower(): c for c in df.columns}
    rename = {}
    for canon, variants in CANON.items():
        found = None
        for v in variants:
            k = v.lower()
            if k in lower_map:
                found = lower_map[k]
                break
        if found and found != canon:
            rename[found] = canon
    if rename:
        df = df.rename(columns=rename)
    # Strong typing
    if "disbursement_date" in df.columns:
        df["disbursement_date"] = pd.to_datetime(df["disbursement_date"], errors="coerce")
    if "snapshot_date" in df.columns:
        df["snapshot_date"] = pd.to_datetime(df["snapshot_date"], errors="coerce").dt.date
    for c in ("disbursement_amount", "outstanding_balance", "dpd", "apr_annual", "term_days"):
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

print("[BOOTSTRAP] normalize_columns() loaded.")

In [ ]:
# =========================
# NOTEBOOK BOOTSTRAP (ABACO)
# =========================

# 1) Restore Python built-ins if shadowed in notebook scope
# (common causes: variables named `len`, `list`, `print`, `sum`, `input`, etc.)
import builtins as _builtins
for _name in ("print", "len", "sum", "min", "max", "list", "dict", "set", "input", "any", "all", "sorted", "map"):
    if _name in globals() and globals()[_name] is not getattr(_builtins, _name, None):
        globals()[_name] = getattr(_builtins, _name)
del _name, _builtins

# 2) Core imports (avoid "NameError: Path is not defined", etc.)
import os
from pathlib import Path
from datetime import datetime

import pandas as pd

# 3) Pandas settings (optional)
pd.set_option("display.max_columns", 30)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}" if abs(x) < 1 else f"{x:,.2f}")

# 4) Optional dependencies: supabase client
# If not installed, the notebook can still run with CSV fallback.
try:
    from supabase import create_client
except Exception:
    create_client = None

print("[BOOTSTRAP] Ready. Built-ins restored, imports loaded.")

In [ ]:
# --- Config + Environment Validation (must run first) ---
import os
REQUIRED_ENV = ["SUPABASE_URL", "SUPABASE_ANON_KEY"]
missing = [k for k in REQUIRED_ENV if not os.getenv(k)]
if missing:
    raise EnvironmentError(f"Missing required env vars: {missing}. Set them before running the notebook.")

OSError: Missing required env vars: ['SUPABASE_URL', 'SUPABASE_ANON_KEY']. Set them before running the notebook.

## 2. Define Financial Analysis Formulas
Implement Python functions for the financial formulas used in client analysis (APR, yield, DPD, rotation, ticket, recurrence, etc.).

In [ ]:
# --- Financial Formulas for Client Analysis ---
import builtins
def weighted_average(series, weights):
    return np.average(series, weights=weights) if builtins.len(series) > 0 else np.nan

def apr_annual_nom(apr_m):
    return apr_m * 12

def apr_annual_eff(apr_m):
    return (1 + apr_m) ** 12 - 1

def yield_per_term(apr_m, term):
    return (1 + apr_m) ** (term / 30) - 1

def rotation(avg_term):
    return 360 / avg_term if avg_term > 0 else np.nan

def line_up(disb_first, disb_last):
    return disb_last / disb_first - 1 if disb_first > 0 else np.nan

def rate_down(apr_first, apr_last):
    return apr_last - apr_first

def tenor_up(term_first, term_last):
    return term_last - term_first

def dpd_proxy(days_realized, term):
    return days_realized - term

def percent_back_to_back(days_between, term):
    return np.mean(days_between <= (term + 15)) if builtins.len(days_between) > 0 else np.nan

## 3. Load Client Data
Load the client dataset from a CSV file or database into a pandas DataFrame.

In [ ]:
# --- Robust Supabase loader with paging and schema fallback ---
from supabase import create_client
import pandas as pd

def load_loans_from_supabase(table_or_view_candidates=None, page_size: int = 5000, max_rows: int = 250000) -> pd.DataFrame:
    """
    Robust Supabase loader with paging. Works only for resources exposed via PostgREST.
    If curated schema isn't exposed, create a public view and use that name.
    """
    if table_or_view_candidates is None:
        table_or_view_candidates = [
            "v_loan_portfolio_daily",      # recommended public view name
            "loan_portfolio_daily",        # alternative
            "curated.loan_portfolio_daily" # will fail if schema not exposed
        ]
    url = os.environ["SUPABASE_URL"]
    key = os.environ["SUPABASE_ANON_KEY"]
    sb = create_client(url, key)
    last_err = None
    for name in table_or_view_candidates:
        rows = []
        offset = 0
        try:
            while True:
                resp = (
                    sb.table(name)
                      .select("*")
                      .range(offset, offset + page_size - 1)
                      .execute()
                )
                batch = resp.data or []
                rows.extend(batch)
                if len(batch) < page_size:
                    break
                offset += page_size
                if len(rows) >= max_rows:
                    break
                time.sleep(0.05)  # gentle throttling
            df = pd.DataFrame(rows)
            if df.empty:
                raise ValueError(f"Loaded 0 rows from '{name}'")
            return normalize_columns(df)
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f"All Supabase table/view candidates failed. Last error: {last_err}")

# --- Data loading: Supabase default, CSV fallback ---
def load_loans_from_csv(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p.resolve()}")
    df = pd.read_csv(p, parse_dates=False)
    return normalize_columns(df)

try:
    loans = load_loans_from_supabase()
    print(f"[INFO] Loaded {len(loans)} loans from Supabase.")
except Exception as e:
    print(f"[WARNING] Supabase load failed: {e}. Trying CSV fallback...")
    try:
        loans = load_loans_from_csv("data/loans.csv")
        print(f"[INFO] Loaded {len(loans)} loans from CSV.")
    except Exception as e2:
        print(f"[ERROR] Could not load loans from Supabase or CSV: {e2}")
        loans = None

if loans is not None:
    display(loans.head())
else:
    print("[ERROR] No loans data available. Please check your data source or configuration.")

[WARNING] Supabase load failed: No module named 'supabase'. Trying CSV fallback...
[ERROR] Could not load loans from Supabase or CSV: CSV not found: /Users/jenineferderas/Documents/abaco-loans-analytics/notebooks/data/loans.csv
[ERROR] No loans data available. Please check your data source or configuration.


## 4. Input Client ID for Analysis
Prompt the user to input a client ID or select one from the dataset for analysis.

In [ ]:
# --- Parameterized client selection (supports papermill/CLI) ---
import os

if loans is None or loans.empty:
    raise ValueError("Loans dataset is empty.")

client_ids = loans["customer_id"].dropna().astype(str).unique()
print(f"[INFO] Total unique clients: {len(client_ids)}")
print(f"[INFO] Sample client IDs: {client_ids[:25]}")

client_id = None
# Priority: injected parameter (e.g., via papermill or os.environ)
if 'client_id' in globals() and client_id is not None:
    pass  # already set
elif os.environ.get('CLIENT_ID'):
    client_id = os.environ['CLIENT_ID']
else:
    try:
        client_id = input("Enter client ID: ").strip()
        if client_id not in set(client_ids):
            print("[WARNING] Client not found; using first sample.")
            client_id = client_ids[0]
    except Exception:
        print("[WARNING] input() not available; using first sample.")
        client_id = client_ids[0]

[ERROR] The loans DataFrame is not loaded. Please run the data loading cell above.


## 5. Apply Formulas to Selected Client
Filter the DataFrame for the selected client ID and apply the defined formulas to compute relevant metrics.

In [ ]:
# --- Client metrics with portfolio benchmarks and APR guard ---
def compute_client_metrics(loans: pd.DataFrame, client_id: str):
    require_columns(loans, ["customer_id", "disbursement_date", "disbursement_amount", "outstanding_balance", "dpd"], "loans")
    df = loans.copy()
    df = df[df["customer_id"].notna()]
    df["customer_id"] = df["customer_id"].astype(str)
    client = df[df["customer_id"] == str(client_id)].copy()
    if client.empty:
        raise ValueError(f"No loans found for client_id={client_id}")
    total_outstanding = df["outstanding_balance"].sum(skipna=True)
    client_outstanding = client["outstanding_balance"].sum(skipna=True)
    par30_bal = client.loc[client["dpd"] >= 30, "outstanding_balance"].sum(skipna=True)
    par60_bal = client.loc[client["dpd"] >= 60, "outstanding_balance"].sum(skipna=True)
    par90_bal = client.loc[client["dpd"] >= 90, "outstanding_balance"].sum(skipna=True)
    avg_term = weighted_avg(client["term_days"], client["disbursement_amount"]) if "term_days" in client.columns else np.nan
    avg_apr_annual = weighted_avg(client["apr_annual"], client["disbursement_amount"]) if "apr_annual" in client.columns else np.nan
    client = client.sort_values("disbursement_date")
    client["days_between"] = client["disbursement_date"].diff().dt.days
    b2b = np.nan
    if "term_days" in client.columns and client["days_between"].notna().any():
        term_ref = float(pd.to_numeric(client["term_days"], errors="coerce").mean())
        b2b = float((client["days_between"] <= (term_ref + 15)).mean())
    results = {
        "Total Disbursed": float(client["disbursement_amount"].sum(skipna=True)),
        "Current Outstanding": float(client_outstanding),
        "Share of Portfolio Outstanding": safe_div(client_outstanding, total_outstanding),
        "Max DPD": float(client["dpd"].max(skipna=True)),
        "Mean DPD": float(client["dpd"].mean(skipna=True)),
        "% Exposure DPD>=30": safe_div(par30_bal, client_outstanding),
        "% Exposure DPD>=60": safe_div(par60_bal, client_outstanding),
        "% Exposure DPD>=90": safe_div(par90_bal, client_outstanding),
        "PAR30 (Balance)": safe_div(par30_bal, client_outstanding),
        "PAR60 (Balance)": safe_div(par60_bal, client_outstanding),
        "PAR90 (Balance)": safe_div(par90_bal, client_outstanding),
        "Avg Ticket": float(client["disbursement_amount"].mean(skipna=True)),
        "Median Days Between": float(client["days_between"].median(skipna=True)) if client["days_between"].notna().any() else np.nan,
        "P75 Days Between": float(client["days_between"].quantile(0.75)) if client["days_between"].notna().any() else np.nan,
        "% Back-to-Back": b2b,
        "Avg Term (days)": float(avg_term) if not np.isnan(avg_term) else np.nan,
        "Rotation (annualized)": safe_div(360.0, avg_term) if not np.isnan(avg_term) else np.nan,
        "APR Annual (weighted)": float(avg_apr_annual) if not np.isnan(avg_apr_annual) else np.nan,
    }
    if len(client) > 1:
        first, last = client.iloc[0], client.iloc[-1]
        results["Line Up (first→last)"] = safe_div(last["disbursement_amount"], first["disbursement_amount"]) - 1
        if "apr_annual" in client.columns:
            results["Rate Delta (bps)"] = (float(last.get("apr_annual", np.nan)) - float(first.get("apr_annual", np.nan))) * 10000
        if "term_days" in client.columns:
            results["Tenor Delta (days)"] = float(last.get("term_days", np.nan)) - float(first.get("term_days", np.nan))
    else:
        results["Line Up (first→last)"] = np.nan
        results["Rate Delta (bps)"] = np.nan
        results["Tenor Delta (days)"] = np.nan
    results_df = pd.DataFrame(results, index=[str(client_id)])
    # --- Portfolio benchmarks ---
    def portfolio_benchmarks(loans: pd.DataFrame) -> dict:
        require_columns(loans, ["outstanding_balance", "dpd"], "loans")
        total = loans["outstanding_balance"].sum(skipna=True)
        par30 = loans.loc[loans["dpd"] >= 30, "outstanding_balance"].sum(skipna=True)
        par90 = loans.loc[loans["dpd"] >= 90, "outstanding_balance"].sum(skipna=True)
        return {
            "Portfolio Outstanding": float(total),
            "Portfolio PAR30": safe_div(par30, total),
            "Portfolio PAR90": safe_div(par90, total),
        }
    bench = portfolio_benchmarks(loans)
    for k, v in bench.items():
        results_df.loc[str(client_id), k] = v
    # --- APR guard ---
    if "apr_annual" in loans.columns and loans["apr_annual"].dropna().between(0, 3).mean() < 0.8:
        print("[WARNING] apr_annual has values outside [0,3]. Verify if rates are in % or decimals.")
    return results_df, client

# --- Usage: compute metrics ---
results_df, client_loans = compute_client_metrics(loans, client_id)
display(results_df)
display(client_loans.head())

[ERROR] The loans DataFrame is not loaded. Please run the data loading cell above.


In [ ]:
# --- Output classification: Watchlist / Run-off / Top Clients ---
def classify_client(results: pd.Series) -> dict:
    par90 = results.get("PAR90 (Balance)", np.nan)
    share = results.get("Share of Portfolio Outstanding", np.nan)
    dpd_max = results.get("Max DPD", np.nan)
    if pd.notna(par90) and par90 >= 0.10:
        tier = "RUN_OFF"
    elif pd.notna(dpd_max) and dpd_max >= 60:
        tier = "WATCHLIST"
    elif pd.notna(share) and share >= 0.05:
        tier = "TOP_CLIENT"
    else:
        tier = "STANDARD"
    return {"tier": tier}

cls = classify_client(results_df.loc[str(client_id)])
results_df.loc[str(client_id), "Client Tier"] = cls["tier"]
display(results_df)

## 6. Display Analysis Results
Present the computed metrics and analysis results for the selected client in a clear, tabular format.

In [ ]:
# --- Matplotlib-only visuals for client analysis (robust, CI-friendly) ---
import matplotlib.pyplot as plt

def plot_client(client_loans: pd.DataFrame, client_id: str):
    df = client_loans.sort_values("disbursement_date").copy()
    plt.figure(figsize=(10,4))
    plt.hist(df["dpd"].dropna(), bins=20)
    plt.title(f"DPD Distribution — {client_id}")
    plt.xlabel("DPD (days)")
    plt.ylabel("Loans")
    plt.show()
    if "apr_annual" in df.columns:
        plt.figure(figsize=(10,4))
        plt.plot(df["disbursement_date"], df["apr_annual"], marker="o")
        plt.title(f"APR Annual Trend — {client_id}")
        plt.ylabel("APR annual")
        plt.xlabel("Disbursement date")
        plt.show()

# --- Usage ---
plot_client(client_loans, client_id)

# --- Minimal, safe exports ---
from pathlib import Path
from datetime import datetime
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
results_df.to_csv(out_dir / f"client_{client_id}_analysis_{ts}.csv", index=True)
results_df.to_excel(out_dir / f"client_{client_id}_analysis_{ts}.xlsx", index=True)
print(f"[INFO] Exported to {out_dir.resolve()}")

NameError: name 'client_loans' is not defined

In [ ]:
# --- Improved Effective Yield (cash-based) + Annualization ---
import pandas as pd

def add_effective_yield(client_loans: pd.DataFrame, results_df: pd.DataFrame) -> pd.DataFrame:
    df = client_loans.copy()
    required = ["disbursement_amount", "interest_cash", "fees_cash", "other_cash", "rebates_cash"]
    if not all(c in df.columns for c in required):
        results_df.loc[:, "Effective Yield (term, mean)"] = np.nan
        results_df.loc[:, "Effective Yield (term, weighted)"] = np.nan
        results_df.loc[:, "Effective Yield (annualized, weighted)"] = np.nan
        return results_df
    disb = pd.to_numeric(df["disbursement_amount"], errors="coerce")
    disb = disb.where(disb > 0, np.nan)
    cash = (
        pd.to_numeric(df["interest_cash"], errors="coerce").fillna(0)
        + pd.to_numeric(df["fees_cash"], errors="coerce").fillna(0)
        + pd.to_numeric(df["other_cash"], errors="coerce").fillna(0)
        - pd.to_numeric(df["rebates_cash"], errors="coerce").fillna(0)
    )
    df["yield_term"] = cash / disb
    results_df.loc[:, "Effective Yield (term, mean)"] = float(df["yield_term"].mean(skipna=True))
    results_df.loc[:, "Effective Yield (term, weighted)"] = weighted_avg(df["yield_term"], df["disbursement_amount"])
    # Optional: annualize if term_days is available (360-day convention)
    if "term_days" in df.columns:
        term = pd.to_numeric(df["term_days"], errors="coerce")
        m = df["yield_term"].notna() & term.notna() & (term > 0)
        if m.any():
            annualized = (1 + df.loc[m, "yield_term"]) ** (360.0 / term.loc[m]) - 1
            results_df.loc[:, "Effective Yield (annualized, weighted)"] = float(
                np.average(annualized, weights=pd.to_numeric(df.loc[m, "disbursement_amount"], errors="coerce"))
            )
        else:
            results_df.loc[:, "Effective Yield (annualized, weighted)"] = np.nan
    else:
        results_df.loc[:, "Effective Yield (annualized, weighted)"] = np.nan
    return results_df

# --- Robust EEFF Parsing + Deterministic Row Picking ---
_NUM_CLEAN = re.compile(r"[^\d\.\-\(\)]+")

def parse_numeric(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    if isinstance(x, (int, float, np.number)):
        return float(x)
    s = str(x).strip()
    if not s:
        return None
    neg = s.startswith("(") and s.endswith(")")
    s = s[1:-1] if neg else s
    s = s.replace(",", "")
    s = _NUM_CLEAN.sub("", s)
    try:
        v = float(s)
        return -v if neg else v
    except Exception:
        return None

def find_best_row(df: pd.DataFrame, patterns_ranked):
    for pat in patterns_ranked:
        rx = re.compile(pat, re.IGNORECASE)
        for i, row in df.iterrows():
            if any(rx.search(str(cell)) for cell in row.values if pd.notna(cell)):
                return i
    return None

def extract_row_value(df: pd.DataFrame, row_idx: int, prefer_col: int | None = None):
    row = df.iloc[row_idx]
    if prefer_col is not None and 0 <= prefer_col < df.shape[1]:
        v = parse_numeric(row.iloc[prefer_col])
        if v is not None:
            return v
    candidates = []
    for v in row.values:
        n = parse_numeric(v)
        if n is not None:
            candidates.append(n)
    return candidates[-1] if candidates else None

def compute_nimal_from_eeff(excel_path: str, sheet: str = "PYG", prefer_col: int | None = None) -> dict:
    pyg = pd.read_excel(excel_path, sheet_name=sheet, header=None)
    income_row = find_best_row(
        pyg,
        patterns_ranked=[
            r"\bingresos?\s+financier",   # highest priority
            r"\bingresos?\b",             # fallback
        ],
    )
    cost_row = find_best_row(
        pyg,
        patterns_ranked=[
            r"\bcosto\s+de\s+fond",       # funding cost preferred
            r"\bcosto\s+direct",          # fallback
            r"\bcosto\b|\bcost\b",
        ],
    )
    loss_row = find_best_row(
        pyg,
        patterns_ranked=[
            r"\bp[eé]rdid(as)?\s+credit", # credit losses preferred
            r"\bcastig",                  # charge-offs
            r"\bp[eé]rdid|loss",
        ],
    )
    if income_row is None or cost_row is None or loss_row is None:
        return {
            "error": "Could not locate income/cost/loss rows reliably in EEFF.",
            "income_row": income_row,
            "cost_row": cost_row,
            "loss_row": loss_row,
        }
    income = extract_row_value(pyg, income_row, prefer_col=prefer_col)
    cost = extract_row_value(pyg, cost_row, prefer_col=prefer_col)
    loss = extract_row_value(pyg, loss_row, prefer_col=prefer_col)
    if income is None or cost is None or loss is None:
        return {
            "error": "Matched rows but could not parse numeric values (check EEFF formatting).",
            "income": income,
            "direct_cost": cost,
            "losses": loss,
            "income_row": income_row,
            "cost_row": cost_row,
            "loss_row": loss_row,
        }
    nimal_cost = safe_div(income - loss, cost)
    return {
        "income": float(income),
        "direct_cost": float(cost),
        "losses": float(loss),
        "nimal_on_cost": float(nimal_cost) if nimal_cost == nimal_cost else None,
        "income_row": int(income_row),
        "cost_row": int(cost_row),
        "loss_row": int(loss_row),
        "prefer_col": prefer_col,
        "notes": "NIMAL(on cost) = (income - losses) / direct_cost",
    }

# --- AUM From Loan Tape as a True Average (Windowed) ---
from datetime import datetime

def avg_aum_from_loans(
    loans: pd.DataFrame,
    date_col: str = "snapshot_date",
    window_days: int = 30,
    as_of: str | None = None
) -> float:
    if "outstanding_balance" not in loans.columns:
        raise ValueError("Missing outstanding_balance in loans")
    if date_col not in loans.columns:
        return float(loans["outstanding_balance"].sum(skipna=True))
    df = loans[[date_col, "outstanding_balance"]].copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df["outstanding_balance"] = pd.to_numeric(df["outstanding_balance"], errors="coerce")
    df = df.dropna(subset=[date_col, "outstanding_balance"])
    if df.empty:
        return float(loans["outstanding_balance"].sum(skipna=True))
    as_of_dt = pd.to_datetime(as_of) if as_of else df[date_col].max()
    start_dt = as_of_dt - pd.Timedelta(days=window_days - 1)
    w = df[(df[date_col] >= start_dt) & (df[date_col] <= as_of_dt)]
    if w.empty:
        return float(loans["outstanding_balance"].sum(skipna=True))
    daily = w.groupby(date_col)["outstanding_balance"].sum()
    return float(daily.mean())

# --- Usage Block (Upgraded) ---
results_df = add_effective_yield(client_loans, results_df)
display(results_df)

eeff = compute_nimal_from_eeff(
    "../data/EEFF ABACO CONSOLIDADO - ESTADO DE RESULTADOS - SEP-25.xlsx",
    sheet="PYG",
    prefer_col=None  # set an explicit column index once you identify the “latest period” column
)

if "error" not in eeff:
    aum = avg_aum_from_loans(loans, date_col="snapshot_date", window_days=30)
    eeff["avg_aum_used"] = aum
    eeff["nimal_on_aum"] = safe_div(eeff["income"] - eeff["losses"], aum)

print(eeff)

NameError: name 'client_loans' is not defined

In [ ]:
# --- Restore Python built-ins for notebook robustness ---
all = builtins.all
any = builtins.any
isinstance = builtins.isinstance
Exception = builtins.Exception
ValueError = builtins.ValueError
dict = builtins.dict
print = builtins.print
input = builtins.input

In [ ]:
# --- Production-grade risk alerts (robust, structured) ---
def anchor_concentration_alert(invoices: pd.DataFrame, client_id: str, threshold: float = 0.4) -> dict:
    require_columns(invoices, ["customer_id", "anchor_id", "invoice_amount"], "invoices")
    df = invoices.copy()
    df["customer_id"] = df["customer_id"].astype(str)
    sub = df[df["customer_id"] == str(client_id)]
    if sub.empty:
        return {"alert": False, "reason": "no_invoices"}
    total = sub["invoice_amount"].sum()
    if total <= 0:
        return {"alert": False, "reason": "zero_total_amount"}
    shares = sub.groupby("anchor_id")["invoice_amount"].sum() / total
    anchor = shares.idxmax()
    share = float(shares.max())
    return {"alert": share > threshold, "anchor_id": anchor, "share": share, "threshold": threshold}

def credit_line_usage_alert(credit_used, credit_limit, threshold=0.9):
    usage = safe_div(credit_used, credit_limit)
    return {"alert": usage > threshold, "usage": usage, "threshold": threshold}

def extended_term_alert(loans: pd.DataFrame, client_id: str, threshold=90):
    require_columns(loans, ["customer_id", "term_days"], "loans")
    df = loans.copy()
    df["customer_id"] = df["customer_id"].astype(str)
    client_loans = df[df["customer_id"] == str(client_id)]
    long_terms = client_loans[client_loans["term_days"] > threshold]
    return {"alert": not long_terms.empty, "long_terms": long_terms}

def apr_gap_alert(loans: pd.DataFrame, client_id: str, bps_gap=0.05):
    require_columns(loans, ["customer_id", "apr_annual"], "loans")
    df = loans.copy()
    df["customer_id"] = df["customer_id"].astype(str)
    client_loans = df[df["customer_id"] == str(client_id)]
    if "apr_effective" in client_loans.columns and "apr_annual" in client_loans.columns:
        gap = client_loans["apr_annual"] - client_loans["apr_effective"]
        alert = (gap > bps_gap).any()
        return {"alert": alert, "gap": gap}
    return {"alert": False, "gap": None}

# --- Example usage for a client (requires invoices, credit_used, credit_limit) ---
# invoices: DataFrame with ['customer_id', 'anchor_id', 'invoice_amount']
# loans: DataFrame with ['customer_id', 'term_days', 'apr_annual', 'apr_effective']
# credit_used, credit_limit: numeric values for the client
#
# alert1 = anchor_concentration_alert(invoices, client_id)
# if alert1["alert"]:
#     print(f"Alert: >{int(alert1['threshold']*100)}% of invoices with anchor {alert1['anchor_id']} ({alert1['share']:.1%}). Recommend diversifying and limiting exposure.")
#
# alert2 = credit_line_usage_alert(credit_used, credit_limit)
# if alert2["alert"]:
#     print(f"Alert: Credit line usage >{int(alert2['threshold']*100)}% ({alert2['usage']:.1%}). Review capacity and monitor liquidity.")
#
# alert3 = extended_term_alert(loans, client_id)
# if alert3["alert"]:
#     print(f"Alert: Payment terms >90 days detected. Negotiate shorter terms.")
#     display(alert3["long_terms"][['term_days']])
#
# alert4 = apr_gap_alert(loans, client_id)
# if alert4["alert"]:
#     print(f"Alert: Effective APR >500 bps lower than contractual. Review pricing and adjust rates.")
#     display(alert4["gap"])

# Credit Policy Guidelines and Actions to Optimize Risk-Return

**Advance Rate by Risk:**
- AAA/Top Anchors: Advance rate 90–95% of invoice value.
- Medium risk: Advance 80–85%.
- High risk/unverified: Advance 70–75%.
- _Action:_ Automatically adjust advance rate based on anchor/payer and client rating. Retain more for payers/segments with a history of late payments or no track record.

**Limits per Anchor (Payer):**
- Maximum limit: 20% of the portfolio per anchor; 10% for medium anchors or lower ratings.
- _Action:_ Monitor concentration and reject new operations that exceed the threshold, or seek co-financiers/insurance.

**Limits per Client (Supplier):**
- Maximum limit: 10% of Abaco's equity or an absolute amount based on loss absorption capacity.
- Graduation: Lower lines for B–C clients and minimums for D–E.
- _Action:_ Automate alerts and line reviews as balances grow.

**Minimum Pricing by Risk Bucket:**
- A: Minimum APR 25%, commission 1–2%.
- B–C: Minimum APR 30%, standard commission (~2%).
- D–E: Minimum APR 35–40%, commission 3%+.
- F–H: APR ≥50% or case by case.
- _Action:_ Standardize minimums in the system to avoid price wars and protect margin.

**Renewal/Evergreen Policies:**
- Mandatory pause after 6 months of continuous use, force liquidation and requalification.
- Condition: Keep DSO under control for evergreen continuity.

---

## Actions by Active Client
- **Top Clients:** Controlled upsell, increase line if using <70%, offer better terms, standard monitoring.
- **Watchlist Clients:** No upsell, reinforce collections, close monitoring, require monthly financial info.
- **Run-off Clients:** No new operations, focus on recovery, document lessons learned.
- **New Potentials:** Prospect clones of the best, pre-approve guidelines for ideal profiles.
- **Payers with Red Flags:** Limit/discontinue financing, monitor news, restricted lists.

---

## Ideal Client Profile
Medium-sized company, strong financial and credit history, supplier to several large corporates, regular and disciplined use of the line, good documentation and transparency.

---

## Conclusion
Implementing these policies and actions allows:
- Maximizing income with top clients (more safe business, better rates).
- Minimizing losses with risky clients (limits, pricing, monitoring).
- Strengthening risk culture and portfolio stability.

# Net Interest Margin After Loss (NIMAL)

**Definition:** Abaco's net financial margin after deducting direct funding costs and credit losses (realized or provisioned).

**Formulas:**
- **NIMAL (on cost):**
  $$(\text{Net financial income} - \text{losses}) / \text{Direct cost}$$
- **NIMAL (on AUM):**
  $$(\text{Net financial income} - \text{losses}) / \text{Average AUM}$$

**Executive Interpretation:**
- **2024:** NIMAL after loss (on direct cost): ~80.2%  
  For every USD 1 of direct cost (funding + credit), Abaco generates ~0.80 USD of net margin after losses.
- **2025 YTD:** NIMAL after loss (on direct cost): ~66.6%  
  The margin remains high, but drops vs 2024 due to higher marginal funding cost, scaling of structure, and greater weight of long terms/risk.

**Connection with Portfolio and KPIs:**
- **NIMAL_AUM:**
  $$(\text{Interest income} + \text{fees} - \text{funding cost} - \text{credit losses}) / \text{Average AUM}$$
- **Relation to Effective APR:**
  NIMAL after loss = Effective APR – funding cost – realized loss rate.

**For Investors:**
- NIMAL is a key KPI to show the real profitability of the portfolio after funding and risk.
- 2024: ~80.2% on direct cost.
- 2025 YTD: ~66.6% on direct cost.

_These ratios are ready to be shown as “NIMAL (post-loss)” in the deck and can be aligned with the effective APR and other KPIs from your loan tape._

In [ ]:
# NIMAL_AUM calculation using EEFF and portfolio data
import pandas as pd
import os
import builtins

# Path to consolidated EEFF file (adjust as needed)
eeff_path = '../data/EEFF ABACO CONSOLIDADO - ESTADO DE RESULTADOS - SEP-25.xlsx'

if not os.path.exists(eeff_path):
    builtins.print(f"[ERROR] File not found: {eeff_path}\nPlease check the path and filename.")
else:
    pyg = pd.read_excel(eeff_path, sheet_name='PYG', header=None)

    # Extract income, direct cost, and losses (adjust rows/columns as needed)
    # Example: search for row labels if available, else use index
    def find_row_index(df, label):
        for i, row in df.iterrows():
            if builtins.any(label.lower() in str(cell).lower() for cell in row):
                return i
        return None

    idx_ingresos = find_row_index(pyg, 'ingreso') or 11
    idx_costo = find_row_index(pyg, 'costo') or 12
    idx_perdidas = find_row_index(pyg, 'pérdida') or 13

    ingresos = pyg.iloc[idx_ingresos, 2]
    costo_directo = pyg.iloc[idx_costo, 2]
    perdidas = pyg.iloc[idx_perdidas, 2]

    # Calculate average AUM from available data (preferably from EEFF or loan tape)
    # Example: if you have aum values in the EEFF, extract them; else, prompt user
    aum_col = None
    for col in builtins.range(pyg.shape[1]):
        if 'aum' in str(pyg.iloc[0, col]).lower():
            aum_col = col
            break

    if aum_col is not None:
        aum_inicio = pyg.iloc[1, aum_col]
        aum_fin = pyg.iloc[-1, aum_col]
        aum_prom = (aum_inicio + aum_fin) / 2 if pd.notnull(aum_inicio) and pd.notnull(aum_fin) else None
    else:
        builtins.print('[WARNING] AUM columns not found in EEFF. Please provide actual AUM values.')
        aum_prom = None

    # NIMAL on cost and on AUM
    nimal_costo = (ingresos - perdidas) / costo_directo if costo_directo else None
    nimal_aum = (ingresos - perdidas) / aum_prom if aum_prom else None

    builtins.print(f"NIMAL (on direct cost): {nimal_costo:.2%}" if nimal_costo is not None else '[ERROR] NIMAL (on direct cost) could not be calculated.')
    builtins.print(f"NIMAL (on average AUM): {nimal_aum:.2%}" if nimal_aum is not None else '[ERROR] NIMAL (on average AUM) could not be calculated.')

[ERROR] File not found: ../data/EEFF ABACO CONSOLIDADO - ESTADO DE RESULTADOS - SEP-25.xlsx
Please check the path and filename.
